In [1]:
import roboticstoolbox as rtb
import spatialmath as sm
import numpy as np
from swift import Swift
from roboticstoolbox import *
import spatialgeometry as sg
from typing import Tuple
from math import pi
import rospy
from kortex_driver.msg import Base_JointSpeeds
from tf.transformations import euler_from_quaternion

class GEN3_LITE(ERobot):

    def __init__(self):

        # links, name, urdf_string, urdf_filepath = self.URDF_read("/home/riot/kinova_gen3_lite/src/ros_kortex/kortex_description/arms/gen3_lite/6dof/urdf/gen3_lite_macro.xacro")
        links, name, urdf_string, urdf_filepath = self.URDF_read("/home/riot/kinova_gen3_lite/src/ros_kortex/kortex_description/robots/gen3_lite_gen3_lite_2f.xacro")
    
        super().__init__(
            links,
            name='gen3-lite',
            urdf_string=urdf_string,
            urdf_filepath=urdf_filepath,
            manufacturer="Kinova",
            gripper_links=links[7]
        )

        self.home = np.array([0   * pi/180, 
                              -16 * pi/180, 
                              75  * pi/180, 
                              0   * pi/180, 
                              -60 * pi/180, 
                              0   * pi/180])
        
        self.qr = np.zeros(6)
        
        
        self.pre = np.array([-48 * pi/180, 
                             38  * pi/180, 
                             137 * pi/180, 
                             94  * pi/180, 
                             74  * pi/180, 
                             39  * pi/180])
        
        
        
        self.addconfiguration(
            "qr", np.array([0, 0, 0, 0, 0, 0])
            
        )
        
        self.addconfiguration(
            "home", np.array([0   * pi/180, 
                              -16 * pi/180, 
                              75  * pi/180, 
                              0   * pi/180, 
                              -60 * pi/180, 
                              0   * pi/180])
            
        )
        
        self.addconfiguration(
            "pre", np.array([-48 * pi/180, 
                             38  * pi/180, 
                             137 * pi/180, 
                             94  * pi/180, 
                             74  * pi/180, 
                             39  * pi/180])
            
        )
        
        
        
        self.qdlim = np.array(
            [1, 1, 1, 1, 1, 1.57]
        )




In [2]:
kinova_lite = GEN3_LITE()
# kinova_lite.q = kinova_lite.configs["qr"]
print(kinova_lite)

ERobot: gen3-lite (by Kinova), 6 joints (RRRRRR), 1 gripper, dynamics, geometry, collision
┌─────┬────────────────────┬───────┬──────────────────┬───────────────────────────────────────────────────┐
│link │        link        │ joint │      parent      │                ETS: parent to link                │
├─────┼────────────────────┼───────┼──────────────────┼───────────────────────────────────────────────────┤
│   0 │ base_link          │       │ BASE             │ SE3()                                             │
│   1 │ shoulder_link      │     0 │ base_link        │ SE3(0, 0, 0.1283) ⊕ Rz(q0)                        │
│   2 │ arm_link           │     1 │ shoulder_link    │ SE3(0, -0.03, 0.115; 90°, -0°, 0°) ⊕ Rz(q1)       │
│   3 │ forearm_link       │     2 │ arm_link         │ SE3(0, 0.28, 0; 180°, -0°, 0°) ⊕ Rz(q2)           │
│   4 │ lower_wrist_link   │     3 │ forearm_link     │ SE3(0, -0.14, 0.02; 90°, -0°, 0°) ⊕ Rz(q3)        │
│   5 │ upper_wrist_link   │     4 │ lower_wr

In [24]:
env = Swift()

env.launch(realtime=True, browser="notebook")

# We can then add our robot to the simulator envionment
env.add(kinova_lite)

tool_frame = (
    kinova_lite.fkine(kinova_lite.q) * sm.SE3.Tz(0.13) * sm.SE3.Rz(1.571)
) 



# end-effector axes
ee_axes = sg.Axes(0.2, pose=tool_frame)

# goal axes
goal_axes = sg.Axes(0.1)

# Add the axes to the environment
 

In [25]:
# We can set the pose of the shape using the `.T` attribute.
# This pose can be either a spatialmat SE3 or a 4x4 ndarray
ee_axes.T = kinova_lite.fkine(kinova_lite.q)

# Set the goal axes to something along the x axis
goal_axes.T = sm.SE3.Trans(0.6, 0.0, 0.3)


kinova_lite.q = kinova_lite.pre

# Step the environment with a time step of 0 seconds


# Set all joint velocities to 0.1 rad/s
kinova_lite.qd = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1])


env.add(ee_axes)
env.add(goal_axes)
# # Step the environment 100 times with a time step of 50 milliseconds
# for _ in range(100):
#     env.step(0.05)

2

<br>

<a id='rrmc'></a>

### 3.3 Resolved-Rate Motion Control
---



Resolved-rate motion control (RRMC) is a simple and elegant method to generate straight-line motion of the end effector. RRMC is a direct application of the first-order differential equation

\begin{equation*}
   {^0\bf{\nu}} = {^0\bf{J}}(\bf{q}) \ \bf{\dot{q}}
\end{equation*}

where ${^0(\cdot)}$ denotes the base-frame. RRMC is simply the inverse of the above

\begin{equation*}
    \bf{\dot{q}} = {^0\bf{J}}(\bf{q})^{-1} \ {^0\bf{\nu}}
\end{equation*}

which can only be solved when $\bf{J}(\bf{q})$ is square (and non-singular), which is when the robot has 6 degrees-of-freedom.

For redundant robots there is no unique solution for the above equation. Consequently, the most common approach is to use the Moore-Penrose pseudoinverse

\begin{equation*}
    \bf{\dot{q}} = {^0\bf{J}}(\bf{q})^{+} \ {^0\bf{\nu}}.
\end{equation*}



In [ ]:
from kortex_driver.msg import JointSpeed
from std_msgs.msg import Empty
from copy import deepcopy

def format_speed(qd):
    joint_speeds = []
    
    for i in range(len(qd)):
        joint_speed = deepcopy(JointSpeed())
        joint_speed.joint_identifier = i
        joint_speed.value = qd[i]
        joint_speeds.append(joint_speed)
    
    return joint_speeds            
        
        








rospy.init_node('ok', disable_signals=True)

# Make a new environment and add our robot
env = Swift()
env.launch(realtime=True, browser="notebook")
env.add(kinova_lite)

# Setup publisher
vel_pub = rospy.Publisher('/kinova_gen3_lite/in/joint_velocity', Base_JointSpeeds, queue_size=10)
stop_pub = rospy.Publisher('/kinova_gen3_lite/in/stop', Empty, queue_size=1)


# Create message type and populate
vel_msg = Base_JointSpeeds()
stop_msg = Empty()



# Change the robot configuration to the ready position
kinova_lite.q = kinova_lite.home

# Step the sim to view the robot in this configuration
env.step(0)

# Specify our desired end-effector velocity
ev = [0.0, -0.1, 0.0, 0.0, 0.0, 0.0]

# Specify our timestep
dt = 0.05

# rate = rospy.Rate(100)
# print(vel_msg)
# vel_pub.publish(vel_msg)


# Run the simulation for 5 seconds
for _ in range(50):
    
    # Work out the manipulator Jacobian using the current robot configuration
    J = kinova_lite.jacob0(kinova_lite.q)

    # Since the Panda has 7 joints, the Jacobian is not square, therefore we must
    # use the pseudoinverse (the pinv method)
    J_pinv = np.linalg.inv(J)

    # Calculate the required joint velocities and apply to the robot
    kinova_lite.qd = J_pinv @ ev 
    
    vel_msg.joint_speeds = format_speed(kinova_lite.qd)
    vel_pub.publish(vel_msg)
    
    # Step the simulator by dt seconds
    env.step(dt)
    
    
stop_pub.publish(stop_msg)

<br>

<a id='pbs'></a>

### 3.4 Position based Servoing
---

A more useful application of RRMC is to employ it in a closed-loop pose controller which we denote position-based servoing (PBS). Using this method we can get the end-effector to travel in a straight line, in the robot's task space, towards some desired end-effector pose. The PBS scheme relies on an error vector which represents the translation and rotation from the end-effector's current pose to the desired pose

\begin{equation*}
    \bf{e} =
    \begin{pmatrix}
        \tau\left({^0\bf{T}}_{e^*}\right) - \tau\left({^0\bf{T}}_{e}\right) \\
        \alpha
        \left(
            \rho\left({^0\bf{T}}_{e^*}\right)
            \rho\left({^0\bf{T}}_{e}\right)^\top
        \right)
    \end{pmatrix} \in \mathbb{R}^6
\end{equation*}

where the top three rows correspond to the translational error in the world frame, the bottom three rows correspond to the rotational error in the world frame, ${^0\bf{T}}_{e}$ is the forward kinematics of the robot which represents the end-effector pose in the base frame of the robot, ${^0\bf{T}}_{e^*}$ is the desired end-effector pose in the base frame of the robot ($\cdot^*$ denotes desired not actual), and $\alpha(\cdot) : \bf{SO}(3) \mapsto \mathbb{R}^3$ transforms a rotation matrix to its Euler vector equivalent. 

<br>

<a id='rtb'></a>
## Goal Pose is Fixed
### RPY Or Angle-axis method (error in the base frame)
- RPY is the error in the end-effector frame, so you use jacobe() to calculate the jacobian, which is in end-effector frame
- Angle-axis is the error in the base frame, so you use jacob0() to calculate the jacobian, which is the base frame way of calculation
---

In [26]:
# Make a new environment and add our robot
env = Swift()
env.launch(realtime=True, browser="notebook")
env.add(kinova_lite)
env.add(ee_axes)
env.add(goal_axes)

# Change the robot configuration to the ready position
kinova_lite.q = kinova_lite.pre

# Step the sim to view the robot in this configuration
env.step(0)

# A variable to specify when to break the loop
arrived = False

# Specify the gain for the p_servo method
gain = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

# Specify our timestep
dt = 0.05

tool_frame = (
    kinova_lite.fkine(kinova_lite.q) * sm.SE3.Tz(0.13) * sm.SE3.Rz(1.571)
) 


r,p,y = euler_from_quaternion([0.0053117054792, 
                               0.7192909056869605, 
                               -0.6941433696979334,
                               0.027520194136891465])



# Tep = kinova_lite.fkine(kinova_lite.q) * sm.SE3.Trans([0.1120190208231396, 
#                                                       -0.031112418216675628, 
#                                                       0.19582804495254266]) 



Tep = sm.SE3.Trans([0.3178297, -0.1433714648, 0.04601498014]) *sm.SE3.RPY(r, p, y) 




Tep = Tep.A

# Set the goal axes to Tep
goal_axes.T = Tep

# Run the simulation until the robot arrives at the goal
while not arrived:

    # Work out the base frame manipulator Jacobian using the current robot configuration
    J = kinova_lite.jacobe(kinova_lite.q)

    # The end-effector pose of the panda (using .A to get a numpy array instead of an SE3 object)
    Te = kinova_lite.fkine(kinova_lite.q)* sm.SE3.Tz(0.13) * sm.SE3.Rz(1.571)
    
    # Since the Panda has 7 joints, the Jacobian is not square, therefore we must
    # use the pseudoinverse (the pinv method)
    J_pinv = np.linalg.inv(J)

    # Calculate the required end-effector velocity and whether the robot has arrived
    ev, arrived = rtb.p_servo(Te.A, Tep, gain=gain, threshold=0.001, method='rpy')
    # ev, arrived = rtb.p_servo(Te, Tep, gain=gain, threshold=0.001, method='angle-axis')

    # Calculate the required joint velocities and apply to the robot
    kinova_lite.qd = J_pinv @ ev

    # Update the ee axes
    ee_axes.T = Te

    # Step the simulator by dt seconds
    env.step(dt)

KeyboardInterrupt: 

<br>

<a id='rea'></a>

### 3.7 Dynamic Target Example
---

In [ ]:
# Make a new environment and add our robot
env = Swift()
env.launch(realtime=True)
env.add(kinova_lite)
env.add(ee_axes)
env.add(goal_axes)

# Change the robot configuration to the ready position
kinova_lite.q = kinova_lite.home

# Step the sim to view the robot in this configuration
env.step(0)

# A variable to specify when to break the loop
arrived = False

# Specify the gain for the p_servo method
gain = 2.0 * np.ones(6)

# Specify our timestep
dt = 0.05

# Set the goal axes to Tep
goal_axes.T = sm.SE3.Trans(0.4, -0.2, 0.05) * sm.SE3.Rx(np.pi)

# Make the target move, set its velocity
goal_axes.v = np.array([0.0, 0.02, 0.02, 0.0, 0.0, 0.0])

# Run the simulation for 300 steps
# As the target never stops moving, the robot will never arrive at the goal,
# but it will continuously track towards it
for _ in range(300):

    # Work out the base frame manipulator Jacobian using the current robot configuration
    J = kinova_lite.jacob0(kinova_lite.q)

    # The end-effector pose of the panda (using .A to get a numpy array instead of an SE3 object)
    Te = kinova_lite.fkine(kinova_lite.q).A

    # Since the Panda has 7 joints, the Jacobian is not square, therefore we must
    # use the pseudoinverse (the pinv method)
    J_pinv = np.linalg.inv(J)

    # Calculate the required end-effector velocity
    # Here we feed in the goal_axes.T, which is an SE3, as our goal pose
    ev, _ = rtb.p_servo(Te, goal_axes.T, gain=gain, threshold=0.001, method='angle-axis')

    # Calculate the required joint velocities and apply to the robot
    kinova_lite.qd = J_pinv @ ev

    # Update the ee axes
    ee_axes.T = Te

    # Step the simulator by dt seconds
    env.step(dt)

In [50]:
import roboticstoolbox as rtb
import spatialmath as sm
import numpy as np
import qpsolvers as qp

# Make a new environment and add our robot
env = Swift()
env.launch(realtime=True)
env.add(ee_axes)
env.add(goal_axes)


# Change the robot configuration to the ready position
kinova_lite = GEN3_LITE()
kinova_lite.q = kinova_lite.home
env.add(kinova_lite)

n = 6

Tep = kinova_lite.fkine(kinova_lite.q) * sm.SE3.Trans([0.1120190208231396, -0.031112418216675628, 0.19582804495254266]) * sm.SE3.RPY(r,p,y)
# Tep = sm.SE3.Trans([0.3, 0.3, 0.1]) 


arrived = False

while not arrived:

    # The pose of the Panda's end-effector
    Te: sm.SE3 = kinova_lite.fkine(kinova_lite.q)

    # Transform from the end-effector to desired pose
    eTep: sm.SE3 = Te.inv() * Tep

    # Spatial error
    e = np.sum(np.abs(np.r_[eTep.t, eTep.rpy() * np.pi/180]))

    # Calulate the required end-effector spatial velocity for the robot
    # to approach the goal. Gain is set to 1.0
    v, arrived = rtb.p_servo(Te, Tep, 1.0)

    # Gain term (lambda) for control minimisation
    Y = 0.01

    # Quadratic component of objective function
    Q = np.eye(n + 6)

    # Joint velocity component of Q
    Q[:n, :n] *= Y

    # Slack component of Q
    Q[n:, n:] = (1 / e) * np.eye(6)

    # The equality contraints
    Aeq = np.c_[kinova_lite.jacobe(kinova_lite.q), np.eye(6)]
    beq = v.reshape((6,))

    # The inequality constraints for joint limit avoidance
    Ain = np.zeros((n + 6, n + 6))
    bin = np.zeros(n + 6)

    # The minimum angle (in radians) in which the joint is allowed to approach
    # to its limit
    ps = 0.05

    # The influence angle (in radians) in which the velocity damper
    # becomes active
    pi = 0.9

    # Form the joint limit velocity damper
    Ain[:n, :n], bin[:n] = kinova_lite.joint_velocity_damper(ps, pi, n)

    # Linear component of objective function: the manipulability Jacobian
    c = np.r_[-kinova_lite.jacobm().reshape((n,)), np.zeros(6)]

    # The lower and upper bounds on the joint velocity and slack variable
    lb = -np.r_[kinova_lite.qdlim[:n], 10 * np.ones(6)]
    ub = np.r_[kinova_lite.qdlim[:n], 10 * np.ones(6)]

    # Solve for the joint velocities dq
    qd = qp.solve_qp(Q, c, Ain, bin, Aeq, beq, lb=lb, ub=ub, solver="clarabel")

    # Apply the joint velocities to the Panda
    kinova_lite.qd[:n] = qd[:n]

    # Step the simulator by 50 ms
    env.step(0.05)


/home/riot/kinova_gen3_lite/src/ggcnn_env/lib/python3.8/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted P to scipy.sparse.csc.csc_matrix
For best performance, build P as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/home/riot/kinova_gen3_lite/src/ggcnn_env/lib/python3.8/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted G to scipy.sparse.csc.csc_matrix
For best performance, build G as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/home/riot/kinova_gen3_lite/src/ggcnn_env/lib/python3.8/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted A to scipy.sparse.csc.csc_matrix
For best performance, build A as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
connection handler failed
Traceback (most recent call last):
  File "/home/riot/kinova_gen3_lite/src/ggcnn_env/lib/python3.8/site-packages/websoc

KeyboardInterrupt: 

In [ ]:
import roboticstoolbox as rtb
import spatialmath as sm
import numpy as np
import qpsolvers as qp

# Launch the simulator Swift
env = Swift()
env.launch()

# Create a Panda robot object
panda = rtb.models.Panda()

# Set joint angles to ready configuration
panda.q = panda.qr

# Add the Panda to the simulator
env.add(panda)

# Number of joint in the panda which we are controlling
n = 7

# Set the desired end-effector pose
Tep = panda.fkine(panda.q) * sm.SE3(0.3, 0.2, 0.3)

arrived = False

while not arrived:

    # The pose of the Panda's end-effector
    Te = panda.fkine(panda.q)

    # Transform from the end-effector to desired pose
    eTep = Te.inv() * Tep

    # Spatial error
    e = np.sum(np.abs(np.r_[eTep.t, eTep.rpy() * np.pi/180]))

    # Calulate the required end-effector spatial velocity for the robot
    # to approach the goal. Gain is set to 1.0
    v, arrived = rtb.p_servo(Te, Tep, 1.0)

    # Gain term (lambda) for control minimisation
    Y = 0.01

    # Quadratic component of objective function
    Q = np.eye(n + 6)

    # Joint velocity component of Q
    Q[:n, :n] *= Y

    # Slack component of Q
    Q[n:, n:] = (1 / e) * np.eye(6)

    # The equality contraints
    Aeq = np.c_[panda.jacobe(panda.q), np.eye(6)]
    beq = v.reshape((6,))

    # The inequality constraints for joint limit avoidance
    Ain = np.zeros((n + 6, n + 6))
    bin = np.zeros(n + 6)

    # The minimum angle (in radians) in which the joint is allowed to approach
    # to its limit
    ps = 0.05

    # The influence angle (in radians) in which the velocity damper
    # becomes active
    pi = 0.9

    # Form the joint limit velocity damper
    Ain[:n, :n], bin[:n] = panda.joint_velocity_damper(ps, pi, n)

    # Linear component of objective function: the manipulability Jacobian
    c = np.r_[-panda.jacobm().reshape((n,)), np.zeros(6)]

    # The lower and upper bounds on the joint velocity and slack variable
    lb = -np.r_[panda.qdlim[:n], 10 * np.ones(6)]
    ub = np.r_[panda.qdlim[:n], 10 * np.ones(6)]

    # Solve for the joint velocities dq
    qd = qp.solve_qp(Q, c, Ain, bin, Aeq, beq, lb=lb, ub=ub, solver="clarabel")

    # Apply the joint velocities to the Panda
    panda.qd[:n] = qd[:n]

    # Step the simulator by 50 ms
    env.step(0.05)
